## Importing Data Set

In [204]:

import os
os.chdir("C:\\Users\\derek\\Downloads") 

In [408]:
from __future__ import print_function
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random as rd
import hashlib
import statsmodels.api as sm
from scipy.stats import beta
from statsmodels.base.model import GenericLikelihoodModel
import math
import numpy as np
from scipy.optimize import minimize
import scipy.stats as stats
import time
import random




In [4]:
hashlib.md5("drt140030".encode('utf-8')).hexdigest()

'c37d47d82beb19f7ee5edd077cf7cb16'

In [257]:
df = pd.read_csv('Exam_2_Data.csv',header=None,na_values=" NaN")
df = df.to_numpy()


## Generating Dataset to Test Estimation Methods

In [267]:
##I chose to generate a random dataset using a beta distribution, as a beta distribution is very similar to a kuramaswamy distribution
#used to create the exam dataset. The complication here is that the exam distribution does not have the standard upper and lower 
#bounds for a kuramaswamy/beta distribution (0,1). I change the bounds to be 10,-10 instead of 0,1 by forcing the loc to be 10
# and the scale to be 20 (scale is 10+-10 which is 20).
a = 2
b = 3
x = beta.rvs(a, b, loc=-10, scale=20, size=200)
x


array([-2.15075165, -4.56172188, -3.80414137, -4.3986439 , -6.46589753,
       -4.67422188, -3.15968417, -2.32780881, -8.25128157, -4.12453886,
        1.06270015,  4.40320127, -3.19243695,  2.31614367, -3.69356965,
       -5.69175174, -7.16606102, -7.93158943, -2.11592391, -1.53683383,
        5.51549869,  2.22402431, -5.24704816,  1.94670763,  0.89956168,
       -6.86711487, -0.70759478, -5.45348505, -0.58904293, -7.81747395,
       -1.03829867, -7.06154784, -4.76229631, -6.11318618, -1.49225194,
       -3.63516831, -3.13554837, -1.03250208, -8.40110321,  2.08644753,
       -1.87238539, -3.15501186, -1.10764386, -1.68032022, -5.5439711 ,
       -4.19020413, -5.21593898,  6.05575732, -2.2739713 , -1.42360781,
        4.36875522, -4.45158969, -1.97805824, -4.57901002, -6.44655609,
       -0.1035213 , -3.00541909,  1.12885513,  0.16706776,  5.93228137,
        6.79800065,  0.68623492, -0.45667583, -9.0400124 , -6.10309289,
        0.82889401, -0.12504409, -5.31757359,  1.49975114, -2.34

## Method Testing on Upper/Lower Bounds On Generated Data Set

In [271]:
##The first thing I want to do is get a very rough estimate for the bounds using np.min and np.max.
lbe = np.min(x) #rough estimate is -9.4740, which is close but not super accurate to the true lower bound (-10)
ube = np.max(x) #rough estimate is 7.0558, which is not very close to the true upper bound (10)

#np.min and np.max don't seem to be giving very accurate estimates

-9.474067106541057

In [424]:
#I then use beta.fit in order to estimate the bounds. Beta.fit returns alpha,beta,loc,and scale. The lower bound is equal
#to loc, and the upper bound is equal to loc+scale (because scale is upper bound - lower bound).

a1,b1,loc1,scale1 = beta.fit(x)
loc1 #Lbe = -9.7761, which is quite close to the true value of -10
scale1 #Ube = 8.591 (18.3671 + -9.7761) which is also fairly close to the true upper bound of 10

18.367123263084288

## Bootstrapping to Find Confidence Interval for LB/UB of Generated Data Set

In [274]:
# I then had to find a way to get more accurate bound estimators. I decided to use a bootstrapping function to provide a 95% confidence interval
# which would give me a range that should be more accurate with the dataset. I chose to find the 95% confidence interval of np.min and np.max.
#Although those estimates would be rough, I figured that the confidence interval would at least give a more accurate picture as to
#where the upper and lower bounds would be.


In [42]:
def bootstrap(data, n=1000, func=np.mean):
    simulations = list()
    sample_size = len(data)
    xbar_init = np.mean(data)
    for c in range(n):
        itersample = np.random.choice(data, size=sample_size, replace=True)
        simulations.append(func(itersample))
    simulations.sort()
    def ci(p):
        u_pval = (1+p)/2.
        l_pval = (1-u_pval)
        l_indx = int(np.floor(n*l_pval))
        u_indx = int(np.floor(n*u_pval))
        return(simulations[l_indx],simulations[u_indx])
    return(ci)


In [45]:
boot = bootstrap(x, n=100000, func=np.min)
cintervals = [boot(.95)]
print(cintervals) #The 95% confidence interval for the lower bound is between -9.4118 and -8.6888

[(-9.411814375028051, -8.688876883758637)]


In [46]:
boot = bootstrap(x, n=100000, func=np.max)
cintervals = [boot(.95)]
print(cintervals) #The 95% confidence interval for the upper bound is between 6.4569 and 7.3011

[(6.45693101734307, 7.301121057992798)]


## Testing Beta Fit Method on Test Data Set

In [48]:
a1,b1,loc1,scale1 = beta.fit(df)

C:\Users\derek\Anaconda3\lib\site-packages\scipy\stats\_continuous_distns.py:515: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
C:\Users\derek\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


In [49]:
loc1 #The estimated lower bound of the Exam Dataset is -175.3817


-175.38170956694032

In [51]:
scale1-175.3817 #The estimated upper bound of the Exam Dataset is 10.4609

10.46099494694036

## Bootstrapping for Exam Upper Bound/Lower Bound Confidence Interval

In [53]:
df = df.reshape(200) #I changed the df from (1,200) to (200,) so that the numpy function would work
boot = bootstrap(df, n=100000, func=np.min)
cintervals = [boot(.95)]
print(cintervals) #The 95% confidence interval for the np.min is between -6.9329 and -6.7690

[(-6.932932739, -6.769042552)]


In [54]:
boot = bootstrap(x, n=100000, func=np.max)
cintervals = [boot(.95)]
print(cintervals) #the 95% confidence interval for the np.max is between 6.4569 and 7.3011

[(6.45693101734307, 7.301121057992798)]


## Using Generalized Method of Moments to Estimate A and B parameters on Generated Data Set

In [174]:
#I wanted to use the Generalized Method of Moments to estimate the alpha and beta parameters. I calculate a rough estimate of 
#mu and sigma using np.mean and np.var to plug into the equations found below.
a = 2
b = 3
x = beta.rvs(a, b, loc=-10, scale=20, size=200)






In [155]:
np.mean(x) #value of mu is -1.7896
np.var(x) #value of sigma^2 is 14.7368

14.736826301843603

In [198]:
# Using the Generalized Method of Moments equation to find mu and sigma^2 using alpha and beta, I rearranged and found equations to find alpha and beta:
# A = mu(mu(1-mu)/sigma^2-1)
# B = A(1-mu/mu)

A = -1.7896*(((-1.7896*2.7896)/14.7368)-1) #A estimate is 2.3958. This is fairly close to the true A value (2)
B = 2.3958*(2.7896/-1.7896) #B estimate is -3.7345, which is close to the expected value of B (3)
A

2.395848513865697

## Bootstrapping to get CI for A and B parameters in Generated Dataset

In [167]:
##After calculating A and B using the Generalized Method of Moments, I wanted to provide a 95% confidence interval for 
#both A and B. To do this, I will use a bootstrap to estimate a 95% confidence interval of mu and sigma. I will then use those
# estimates to calculate A and B, with the idea that the lower values of mu and sigma can be used to calculate the lower value
# of A and B 

In [188]:
a = 2
b = 3
x = beta.rvs(a, b, loc=-10, scale=20, size=200)

In [41]:
boot = bootstrap(x, n=10000)
cintervals = [boot(.95)]
print(cintervals)

#The 95% confidence interval for mu is between -2.1109 and -.94

[(-2.110966504660959, -0.9495207737157852)]


In [350]:
boot = bootstrap(x, n=10000, func=np.var)
cintervals = [boot(.95)]
print(cintervals)

#The 95% confidence interval for sigma is between 13.884 and 19.548

[(11.633814258165831, 16.250112129093214)]


In [203]:
#I test the same equations using both the lower values of mu and sigma and the higher values of mu and sigma found 
# A = mu(mu(1-mu)/sigma^2-1)
# B = A(1-mu/mu)

A = -2.410*(((-2.410*3.410)/13.884)-1)
A2 = -1.267*(((-1.267*2.267)/19.5480)-1)
print(A,A2) #95% confidence interval for A of the generated dataset is 3.8365, 1.4531 (true value a=2 falls within CI)
B = 3.8365*(3.410/-2.410)
B2 = 1.4531*(2.267/-1.267)
print(B,B2) #95% confidence interval for |B| of the generated dataset is 5.4284, 2.5999 (true value b=3 falls within CI)

3.836506842408528 1.4531668796296295
-5.428408713692947 -2.5999823993685873


## Using Generalized Method of Moments to Estimate A and B of Test Dataset

In [58]:
df= np.asarray(df)
np.mean(df) #mean is 9.9884
np.var(df) #var is 7.1980

7.19804159584956

In [60]:
# A = mu(mu(1-mu)/sigma^2-1)
# B = A(1-mu/mu)

A = 9.9884*((9.9884*-8.9884)/(7.1980-1)) #Alpha estimate is -144.6847
B = -144.6847*(-8.9884/9.9884) #Beta estimate is 130.1994


-144.68472098727074

## Bootstrapping to get A and B Estimates of Exam Dataset

In [63]:
#I now use the same bootstrapping method that I used in finding the bounds in order to find a 95% confidence interval for mu and sigma to plug
#in the new values of mu and sigma into the previous equation which will provide a 95% confidence interval for the alpha and beta values themselves.
# of alpha and beta estimates.

df = df.reshape(200) #I needed to reshape the data frame to a 1-D array in order for the bootstrap function to work.

boot = bootstrap(df,n=100000,func=np.mean)
cintervals = [boot(.95)]
print(cintervals) #95% confidence interval for the mean is between 9.5634 and 10.3314

[(9.563441337675, 10.3314102479)]


In [64]:
boot = bootstrap(df,n=100000,func=np.var)
cintervals = [boot(.95)]
print(cintervals) #95% confidence interval for the variance is between 1.4962 and 13.9320

[(1.4962011249403326, 13.932095861736878)]


In [68]:
#Lower Bounds of Confidence intervals for A and B estimates
# A = mu(mu(1-mu)/sigma^2-1)
# B = A(1-mu/mu)

A = 9.5634*((9.5634*-8.5634)/(1.4962-1)) #Alpha estimate is -1578.3892
B = -1578.3892*(-8.5634/9.5634) #Beta estimate is 1413.3444


In [70]:
#Higher Bounds of Confidence Intervals for A and B estimates
A = 10.3314*((10.3314*-9.3314)/(13.9320-1)) #Alpha estimate is -77.0192
B = -77.0192*(-9.3302/10.3302) #Beta estimate is 69.5634
B
#The 95% confidence intervals for the Exam Dataset's Alpha and Beta parameters would be:
#Alpha CI = (-1578.3892,-77.0192)
#Beta CI = (1413.3444, 69.5634)

69.56346826198912

## Final Answers

Upper Bound Estimate = 10.4609
Lower Bound Estimate = -175.3817
Upper Bound 95% Confidence Interval = (6.4569,7.3011)
Lower Bound 95% Confidence Interval = (-9.4118,-8.6888)

A Estimate = -144.6847
B Estimate = 130.1994
A Estimate 95% Confidence Interval = (-1578.3892,-77.0192)
B Estimate 95% Confidence Interval = (69.5634,1413.3444)